## Подготовим dataset для стартового baseline

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from notebooks.helpers import save_frame, SESSIONS_FRAME, explore_data_modern
from sklearn.model_selection import train_test_split
from catboost import Pool,CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

Настройки отображения

In [2]:
pd.set_option('display.max_columns', 50)
plt.style.use('ggplot') 

Загрузка данных

In [3]:
try:
    sessions = pd.read_pickle('../data/raw/ga_sessions.pkl')
    hits = pd.read_pickle('../data/raw/ga_hits.pkl')
    print("✅ Данные загружены!")
    print(f"Сессии: {sessions.shape[0]} строк, {sessions.shape[1]} колонок")
    print(f"События: {hits.shape[0]} строк, {hits.shape[1]} колонок")
except Exception as e:
    print(f"❌ Ошибка: {e}")

✅ Данные загружены!
Сессии: 1860042 строк, 18 колонок
События: 15726470 строк, 11 колонок


In [6]:
def save_session(comment):
    save_frame(sessions, SESSIONS_FRAME, comment)

In [5]:
save_session("Свеже загруженный фрейм")

Сохранено: ../data/processed/sessions_20250709_1531.pkl | Лог: ../data/processed/log_sessions.txt


## Посмотрим сырой sessions

In [7]:
display(explore_data_modern(sessions, SESSIONS_FRAME))    


🔍 Анализ датафрейма: sessions


**Получаем все уникальные session_id, которые есть в hits**

In [8]:
valid_session_ids = hits['session_id'].unique()

**Удаляем "висячие" сессии (которых нет в hits)**

In [9]:
sessions = sessions[sessions['session_id'].isin(valid_session_ids)].copy()

**Находим все session_id, где был start_chat**

In [10]:
target_sessions = hits[hits['event_action'] == 'start_chat']['session_id'].unique()

**Добавляем столбец target в sessions (1 если сессия в target_sessions, иначе 0)**

In [11]:
sessions['target'] = sessions['session_id'].isin(target_sessions).astype(int)

**Проверяем баланс классов**

In [12]:
print("Доля сессий с start_chat:", sessions['target'].mean())

Доля сессий с start_chat: 0.01774092431531878


## Заполним пропуски

**Сначала в utm_keyword**
- Пропусков много
- Хотелось удалить
- Но ведь прирост качества даёт, пусть не сильно большой, но тем не менее

**Вартант 1, просто явным значением "not_set"**

In [13]:
sessions['utm_keyword_notset_fill'] = sessions['utm_keyword'].fillna('(not_set)')

**Вариант 2, стырим значение из utm_campaign(сомнительно)**

In [14]:
sessions['utm_keyword_campaign_fill'] = sessions['utm_keyword'].fillna(sessions['utm_campaign'])

**Добавим бинарный флаг**

In [15]:
sessions['has_utm_keyword'] = sessions['utm_keyword'].notna().astype(int)

**Заполним пропуски, без затей**

In [16]:
cat_cols = sessions.select_dtypes(include=['object']).columns  
sessions[cat_cols] = sessions[cat_cols].fillna('unknown')  
sessions = sessions.drop(columns=['session_id'])

**На всякий случай посмотрим на sessions

In [17]:
display(explore_data_modern(sessions, SESSIONS_FRAME))


🔍 Анализ датафрейма: sessions


**Освободим hits**

In [18]:
del hits

In [19]:
save_session("Подготовленный dataset, удалены висячие сессии, добавлен target признак, заполнены пропуски")

Сохранено: ../data/processed/sessions_20250709_1601.pkl | Лог: ../data/processed/log_sessions.txt


**Добавим фичей**

In [20]:
sessions['visit_time_minutes'] = (
    sessions['visit_time'].apply(lambda x: x.hour * 60 + x.minute)
)

In [21]:
sessions['is_returning'] = (sessions['visit_number'] > 1).astype(int)

In [22]:
save_session("Время преобразовали в минуты, добавили бинарный признак 'Поторные визит'")

Сохранено: ../data/processed/sessions_20250709_1602.pkl | Лог: ../data/processed/log_sessions.txt


In [23]:
sessions['brand_tier'] = sessions['device_brand'].map({
    'Apple': 'premium',
    'Samsung': 'premium',
    'Huawei': 'mid',
    'Xiaomi': 'mid'
}).fillna('other')

In [24]:
sessions['frequent_visitor'] = (sessions['visit_number'] >= 3).astype(int)

In [25]:
save_session("Добавили сегментацию брендов, добавили бинарный признак 'Частый визитёр'")

Сохранено: ../data/processed/sessions_20250709_1602.pkl | Лог: ../data/processed/log_sessions.txt


**Стратифицируем данные на train, validation, test**

In [50]:
TEST = 0.2

**Сначала отделим тест сохраним отдельно и ни ни**

In [51]:
X_CROSS, X_TEST, y_temp, Y_test = train_test_split(
    sessions,
    sessions['target'], 
    test_size=TEST, 
    stratify=sessions['target'],
    random_state=42
)

In [52]:
# X_TRAIN, X_temp, y_train, y_temp = train_test_split(
#     sessions.drop(columns=['target']),  # Все фичи
#     sessions['target'],                 # Целевая переменная
#     test_size=1 - TRAIN,                     
#     stratify=sessions['target'],        # Стратификация по таргету
#     random_state=42
# )

X_TRAIN, X_VAL, Y_train, Y_val = train_test_split(
    X_CROSS,
    y_temp,
    test_size=TEST/(1-TEST),
    stratify=y_temp,                   # Стратификация снова
    random_state=42
)

**Проверяем баланс**

In [53]:
print("Доля целевых событий:")
print(f"Train: {Y_train.mean():.3f}")
print(f"Val:   {Y_val.mean():.3f}")
print(f"Test:  {Y_test.mean():.3f}")

Доля целевых событий:
Train: 0.018
Val:   0.018
Test:  0.018


**Сохраняем разделённые данные**

In [ ]:
del sessions

In [54]:
X_TRAIN.to_parquet('../data/datasets/train.parquet', index=False)
X_VAL.to_parquet('../data/datasets/val.parquet', index=False)
X_TEST.to_parquet('../data/datasets/test.parquet', index=False)
X_CROSS.to_parquet('../data/datasets/cross.parquet', index=False)

In [ ]:
print(X_TRAIN.info())
print(X_VAL.info())
print(X_TEST.info())
print(X_CROSS.info())

In [55]:
display(explore_data_modern(X_TRAIN, 'Train'))


🔍 Анализ датафрейма: Train


In [56]:
del y_temp, Y_test, Y_train, Y_val, X_TEST, X_TRAIN, X_VAL, X_CROSS